In [1]:
import pandas as pd
data_file_encode = "gb18030"
with open("../data/watermelon_2.csv", mode = 'r', encoding = data_file_encode) as data_file:
    df = pd.read_csv(data_file)

'''
implementation of CART rely on CART_DT.py
'''
import CART_DT

In [2]:
# dicision tree visualization using pydotplus.graphviz
index_train = [0,1,2,5,6,9,13,14,15,16]

df_train = df.iloc[index_train]
df_test  = df.drop(index_train)

# generate a full tree
root = CART_DT.TreeGenerate(df_train)
CART_DT.DrawPNG(root, "decision_tree_full.png")
print("accuracy of full tree: %.3f" % CART_DT.PredictAccuracy(root, df_test))

# pre-purning
root = CART_DT.PrePurn(df_train, df_test)
CART_DT.DrawPNG(root, "decision_tree_pre.png")
print("accuracy of pre-purning tree: %.3f" % CART_DT.PredictAccuracy(root, df_test))

# # post-puring
root = CART_DT.TreeGenerate(df_train)
CART_DT.PostPurn(root, df_test)
CART_DT.DrawPNG(root, "decision_tree_post.png")
print("accuracy of post-purning tree: %.3f" % CART_DT.PredictAccuracy(root, df_test))

accuracy of full tree: 0.286
accuracy of pre-purning tree: 0.429
accuracy of post-purning tree: 0.429


In [3]:
# print the accuracy
# k-folds cross prediction
accuracy_scores = []
n = len(df.index)
k = 5
for i in range(k):
    m = int(n/k)
    test = []
    for j in range(i*m, i*m+m):
        test.append(j)
        
    df_train = df.drop(test)
    df_test = df.iloc[test]
    root = CART_DT.TreeGenerate(df_train)  # generate the tree
    CART_DT.PostPurn(root, df_test)  # post-purning
    
    # test the accuracy
    pred_true = 0
    for i in df_test.index:
        label = CART_DT.Predict(root, df[df.index == i])
        if label == df_test[df_test.columns[-1]][i]:
            pred_true += 1
            
    accuracy = pred_true / len(df_test.index)
    accuracy_scores.append(accuracy) 

# print the prediction accuracy result
accuracy_sum = 0
print("accuracy: ", end = "")
for i in range(k):
    print("%.3f  " % accuracy_scores[i], end = "")
    accuracy_sum += accuracy_scores[i]
print("\naverage accuracy: %.3f" % (accuracy_sum/k))


accuracy: 1.000  0.667  1.000  0.000  0.333  
average accuracy: 0.600
